In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import skew
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet, Lasso
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
import warnings

# deal with potential warnings
warnings.filterwarnings(action='ignore')

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [4]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                  test.loc[:,'MSSubClass':'SaleCondition']))

In [6]:
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [7]:
train["SalePrice"] = np.log1p(train["SalePrice"])

In [8]:
numeric_feats = data.dtypes[data.dtypes != "object"].index
skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

data[skewed_feats] = np.log1p(data[skewed_feats])

In [9]:
data = pd.get_dummies(data, drop_first=True)

In [10]:
data = data.fillna(data.mean())
scaler = StandardScaler()
data[skewed_feats] = scaler.fit_transform(data[skewed_feats])
data.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.419418,-0.020358,-0.103719,7,5,2003,2003,1.219420,0.783352,-0.362698,...,0,0,0,0,1,0,0,0,1,0
1,-1.120845,0.619103,0.146544,6,8,1976,1976,-0.801770,0.892570,-0.362698,...,0,0,0,0,1,0,0,0,1,0
2,0.419418,0.118440,0.457629,7,5,2001,2002,1.146941,0.658276,-0.362698,...,0,0,0,0,1,0,0,0,1,0
3,0.638691,-0.266348,0.136301,7,5,1915,1970,-0.801770,0.387039,-0.362698,...,0,0,0,0,1,0,0,0,0,0
4,0.419418,0.769612,0.922662,8,5,2000,2000,1.440386,0.758231,-0.362698,...,0,0,0,0,1,0,0,0,1,0


In [11]:
#creating matrices for sklearn:
x_train = data[:train.shape[0]]
x_test = data[train.shape[0]:]
y = pd.DataFrame(train.SalePrice)

#### Linear regression
#### It doesn't need a random state as sklearn uses closed-form solution instead of  gradient descent

In [12]:
model_Linear = LinearRegression()
model_Linear.fit(x_train, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [13]:
print('mean squared error:')
-np.mean(cross_val_score(model_Linear, x_train, y, cv=5, scoring='neg_mean_squared_error'))

mean squared error:


0.025433746615611586

In [14]:
y_pre = model_Linear.predict(x_test).reshape(len(x_test), )

#### Ridge

In [15]:
param_grid = {'alpha':np.arange(9.9, 10.1, 0.01)}
model_Ridge = Ridge(random_state=42)
Ridge_clf = GridSearchCV(model_Ridge, param_grid, cv=5, scoring='neg_mean_squared_error')
Ridge_clf.fit(x_train, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=42, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 9.9 ,  9.91,  9.92,  9.93,  9.94,  9.95,  9.96,  9.97,  9.98,
        9.99, 10.  , 10.01, 10.02, 10.03, 10.04, 10.05, 10.06, 10.07,
       10.08, 10.09])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [16]:
print('Best alpha for Ridge regression is:', Ridge_clf.best_estimator_.alpha)
print('Score for best Ridge model is:', -Ridge_clf.best_score_)

Best alpha for Ridge regression is: 10.089999999999996
Score for best Ridge model is: 0.016426495697956345


In [ ]:
y_pre = Ridge_clf.best_estimator_.predict(x_test).reshape(len(x_test), )

#### Lasso

In [17]:
param_grid = {'alpha':np.arange(0.00068, 0.00071, 0.000001)}
model_Lasso = Lasso(random_state=42)
Lasso_clf = GridSearchCV(model_Lasso, param_grid, cv=5, scoring='neg_mean_squared_error')
Lasso_clf.fit(x_train, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=42,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': array([0.00068, 0.00068, 0.00068, 0.00068, 0.00068, 0.00069, 0.00069,
       0.00069, 0.00069, 0.00069, 0.00069, 0.00069, 0.00069, 0.00069,
       0.00069, 0.0007 , 0.0007 , 0.0007 , 0.0007 , 0.0007 , 0.0007 ,
       0.0007 , 0.0007 , 0.0007 , 0.0007 , 0.00071, 0.00071, 0.00071,
       0.00071, 0.00071])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [18]:
print('Best alpha for Lasso regression is:', Lasso_clf.best_estimator_.alpha)
print('Score for best Lasso model is:', -Lasso_clf.best_score_)

Best alpha for Lasso regression is: 0.0007000000000000005
Score for best Lasso model is: 0.016143514666083943


In [ ]:
y_pre = Lasso_clf.best_estimator_.predict(x_test).reshape(len(x_test), )

#### ElasticNet

In [32]:
model_ENet = ElasticNet(random_state=42)
param_grid = {'alpha':np.arange(0.0009, 0.0019, 0.00001)}
Enet_clf = GridSearchCV(model_ENet, param_grid, cv=5, scoring='neg_mean_squared_error')
Enet_clf.fit(x_train, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=42, selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': array([0.0009 , 0.00091, ..., 0.00188, 0.00189])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [33]:
print('Best alpha for Enet regression is:', Enet_clf.best_estimator_.alpha)
print('Score for best Enet model is:', -Enet_clf.best_score_)

Best alpha for Enet regression is: 0.0011800000000000007
Score for best Enet model is: 0.01610486259429746


In [ ]:
y_pre = Enet_clf.best_estimator_.predict(x_test).reshape(len(x_test), )

#### submission

In [21]:
test_id = pd.read_csv('test.csv').Id
price = np.expm1(y_pre)
solution = pd.DataFrame({"id":test_id, "SalePrice":price})
solution.to_csv("final_submission.csv", index = False)